In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv(r"C:\Users\user\Diabetes-Prediction\expirements\Diabetes_prediction.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Diagnosis
0,2,115.863387,56.410731,24.336736,94.385783,26.455940,0.272682,20.100494,0
1,2,92.490122,70.615520,23.443591,138.652426,23.910167,0.665160,44.912281,0
2,1,88.141469,63.262618,23.404364,149.358082,21.948250,0.676022,48.247873,1
3,2,108.453101,67.793632,20.751580,108.751638,24.209304,0.289636,42.749868,0
4,1,127.849443,94.725685,22.603078,25.269987,32.997477,0.601315,32.797789,0


In [8]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Diagnosis                   0
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               1000 non-null   int64  
 1   Glucose                   1000 non-null   float64
 2   BloodPressure             1000 non-null   float64
 3   SkinThickness             1000 non-null   float64
 4   Insulin                   1000 non-null   float64
 5   BMI                       1000 non-null   float64
 6   DiabetesPedigreeFunction  1000 non-null   float64
 7   Age                       1000 non-null   float64
 8   Diagnosis                 1000 non-null   int64  
dtypes: float64(7), int64(2)
memory usage: 70.4 KB


In [11]:
import hopsworks

project = hopsworks.login()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/516244


In [12]:
api_key = 'Te1uRq1N86UX5tGZ.oSTwrDFOLuUmoybnMHQGVXKMTjoZOzu7VfpIYYvGQWRYEOvMkIvLUFCIKJtpcdf9'
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [13]:
fs.name

'ajinravi_featurestore'

In [6]:
import sys
sys.path.append(r'C:\Users\user\Diabetes-Prediction\src\components')

import os
from dotenv import load_dotenv
from components.hopsworks_connect import FeatureStoreManager
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from joblib import dump

load_dotenv()

hopsworks_api_key_value: str = os.environ.get("HOPSWORKS_API_KEY_VALUE")
print("1")

manager = FeatureStoreManager(hopsworks_api_key_value)
print("2")

users_fg = manager.get_feature_group("diabetecs_data", version=1)
print("3")

query = users_fg.select_all()
print("4")

df = query.read()
print("done")

1
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/516244
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.
2
3
4


Finished: Reading data from Hopsworks, using Hive (3.73s) 
done


In [10]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Diagnosis
0,2,115.863387,56.410731,24.336736,94.385783,26.455940,0.272682,20.100494,0
1,2,92.490122,70.615520,23.443591,138.652426,23.910167,0.665160,44.912281,0
2,1,88.141469,63.262618,23.404364,149.358082,21.948250,0.676022,48.247873,1
3,2,108.453101,67.793632,20.751580,108.751638,24.209304,0.289636,42.749868,0
4,1,127.849443,94.725685,22.603078,25.269987,32.997477,0.601315,32.797789,0
...,...,...,...,...,...,...,...,...,...
995,1,103.496355,41.059401,24.930299,43.675978,25.846062,0.455884,19.905419,0
996,1,60.714150,64.082842,24.689064,111.562745,19.464280,0.250560,44.421874,1
997,0,97.597727,64.122312,22.140339,108.138552,22.733052,0.761463,58.949800,1
998,0,66.775137,55.908797,24.885274,219.521484,31.819187,0.382877,46.928923,0


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
X = df.drop(columns=['Diagnosis'],axis = 0)
y = df['Diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
pipelines = {
    'Random Forest': Pipeline([('scaler', StandardScaler()), ('rf', RandomForestClassifier(random_state=42))]),
    'Logistic Regression': Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression(random_state=42))]),
    'XGBoost': Pipeline([('scaler', StandardScaler()), ('xgb', XGBClassifier(random_state=42))]),
    'Gradient Boosting': Pipeline([('scaler', StandardScaler()), ('gb', GradientBoostingClassifier(random_state=42))]),
    'Support Vector Machine': Pipeline([('scaler', StandardScaler()), ('svc', SVC(random_state=42))])
}

# Step 3: Train models and store accuracies
accuracies = {}
for model_name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies[model_name] = accuracy

# Step 4: Select the best model
best_model = max(accuracies, key=accuracies.get)
best_accuracy = accuracies[best_model]

In [31]:
print(f"The best model is {best_model} with accuracy {best_accuracy:.2f}")

The best model is Logistic Regression with accuracy 0.69


In [32]:
accuracies

{'Random Forest': 0.645,
 'Logistic Regression': 0.685,
 'XGBoost': 0.555,
 'Gradient Boosting': 0.615,
 'Support Vector Machine': 0.685}

In [ ]:
from feature_engine.imputation import MeanMedianImputer

In [17]:
NUM_COLS = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction']

In [36]:
from sklearn.impute import SimpleImputer

diab_pipe = Pipeline([
    ("mean_imputation", SimpleImputer(strategy='mean', missing_values=pd.NA, 
                                      add_indicator=False, fill_value=None, copy=True)),
    
    ('scalar', MinMaxScaler()),

    ('Logistic_Regression',LogisticRegression(random_state=42))
])

In [27]:
diab_pipe.fit(X_train,y_train)

Pipeline(steps=[('mean_imputation', SimpleImputer(missing_values=<NA>)),
                ('scalar', MinMaxScaler()),
                ('Logistic_Regression', LogisticRegression(random_state=42))])

In [29]:
y_pred = diab_pipe.predict(X_test)

# Calculating the accuracy score
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.685
